# Wrangle that Data

<ul>
    <li><a href="#Gather">Gather</a></li>
    <li>
        <a href="#Assess">Assess</a>
        <ul>
            <li><a href="#Tweet-archive">Tweet archive</a></li>
            <li><a href="#Image-predictions">Image predictions</a></li>
            <li><a href="#Extended-tweets">Extended tweets</a></li>
            <li><a href="#Users">Users</a></li>
            <li><a href="#Problems">Problems</a></li>
        </ul>
    </li>
    <li>
        <a href="#Clean">Clean</a>
        <ul>
            <li><a href="#Define">Define</a></li>
            <li><a href="#Act">Act</a></li>
        </ul>
    </li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import tweepy
import os.path
import json
import time
from glob import glob

## Gather

In [2]:
# load secrets
# 'secrets.json' is ignored by git'
with open('secrets.json') as secrets_file:
    secrets = json.load(secrets_file)

In [3]:
tweets_raw = pd.read_csv('twitter-archive-enhanced.csv')

In [4]:
image_predictions_file_name = 'image-predictions.tsv'
image_predictions_source_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [5]:
# make sure image predictions are downloaded
if not os.path.exists(image_predictions_file_name):
    response = requests.get(image_predictions_source_url, stream=True)
    with open(image_predictions_file_name, mode='w', encoding='utf-8') as dest_file:
        for chunk in response.iter_content(decode_unicode=True):
            # filter out keep-alive new chunks
            if chunk:
                dest_file.write(chunk)

In [6]:
image_predictions_raw = pd.read_csv(image_predictions_file_name, sep='\t')

In [7]:
# get twitter API
consumer_key = secrets['twitter']['consumerApiKey']
consumer_secret = secrets['twitter']['consumerSecret']
access_token = secrets['twitter']['accessToken']
access_secret = secrets['twitter']['accessTokenSecret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

twitter_api = tweepy.API(
    auth,
    wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

In [8]:
status_ids = (tweets_raw['expanded_urls']
    .str.extract(r'^http[s]?://twitter\.com/dog_rates/status/(\d+)', expand=False)
    .dropna()
    .drop_duplicates()
    .astype(str))
status_count = len(status_ids)
len(status_ids)

1967

In [9]:
already_downloaded_statuses = (
    pd.Series(os.listdir('extended-statuses'))
        .str.extract(r'^(\d+)\.json$', expand=False)
        .dropna())
len(already_downloaded_statuses)

1967

In [10]:
statuses_to_download = set(status_ids) - set(already_downloaded_statuses)
len(statuses_to_download)

0

In [11]:
print('Pulling twitter statuses.')
i = 0
for status_id in statuses_to_download:
    try:
        status = api.get_status(status_id, tweet_mode='extended')
        with open(f'extended-statuses/{status_id}.json', 'w') as target_file:
            json.dump(status._json, target_file, indent=2)
    except tweepy.RateLimitError as rle:
        print(rle)
        time.sleep(60 * 5)
        status = api.get_status(status_id, tweet_mode='extended')
        with open(f'extended-statuses/{status_id}.json', 'w') as target_file:
            json.dump(status._json, target_file, indent=2)
    except Exception as e:
        print(e)
    i += 1
    if i % 100 == 0:
        print(f'Statuses pulled so far: {i}.')
        time.sleep(60)
print('Done.')

Pulling twitter statuses.
Done.


In [12]:
already_downloaded_statuses = (
    pd.Series(os.listdir('extended-statuses'))
        .str.extract(r'^(\d+)\.json$', expand=False)
        .dropna())
len(already_downloaded_statuses)

1967

In [13]:
extended_tweets_arr = []
users_arr = []
for file_path in glob('extended-statuses/*.json'):
    with open(file_path, 'r', encoding='utf-8') as status_file:
        status = json.load(status_file)
    extended_tweets_arr.append({
        'tweet_id': status['id'],
        'text': status['full_text'],
        'source': status['source'],
        'in_reply_to_status_id': status['in_reply_to_status_id'],
        'in_reply_to_user_id': status['in_reply_to_user_id'],
        'retweet_count': status['retweet_count'],
        'favorite_count': status['favorite_count'],
        'user_id': status['user']['id']})
    users_arr.append({
        'user_id': status['user']['id'],
        'followers_count': status['user']['followers_count'],
        'friends_count': status['user']['friends_count'],
        'listed_count': status['user']['listed_count'],
        'favourites_count': status['user']['favourites_count'],
        'statuses_count': status['user']['statuses_count']})
extended_tweets_raw = pd.DataFrame(extended_tweets_arr)
users_raw = pd.DataFrame(users_arr)

## Assess

### Tweet archive

In [14]:
tweets_raw

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [15]:
tweets_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [16]:
tweets_raw['name'].value_counts()

None         745
a             55
Charlie       12
Cooper        11
Lucy          11
Oliver        11
Penny         10
Tucker        10
Lola          10
Bo             9
Winston        9
Sadie          8
the            8
an             7
Toby           7
Buddy          7
Daisy          7
Bailey         7
Rusty          6
Milo           6
Oscar          6
Stanley        6
Leo            6
Jax            6
Jack           6
Dave           6
Koda           6
Scout          6
Bella          6
very           5
            ... 
Akumi          1
Ember          1
Rodman         1
Sweet          1
Brooks         1
Bobby          1
Lucia          1
Lilah          1
Jareld         1
Ed             1
Scruffers      1
Dobby          1
Henry          1
Shooter        1
Rolf           1
Acro           1
Stark          1
Ziva           1
Ester          1
Rueben         1
Cermet         1
Aqua           1
Odin           1
Champ          1
Mabel          1
Jennifur       1
Philippe       1
Rumble        

In [17]:
not_names = (
    tweets_raw['name']
    .where(lambda n: n.str.slice(0, 1) == n.str.slice(0, 1).str.lower())
    .dropna()
    .unique())
not_names

array(['such', 'a', 'quite', 'not', 'one', 'incredibly', 'mad', 'an',
       'very', 'just', 'my', 'his', 'actually', 'getting', 'this',
       'unacceptable', 'all', 'old', 'infuriating', 'the', 'by',
       'officially', 'life', 'light', 'space'], dtype=object)

In [18]:
tweets_raw['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [19]:
tweets_raw['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [20]:
tweets_raw['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [21]:
tweets_raw['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [22]:
tweets_raw['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [23]:
tweets_raw['tweet_id'].value_counts().head()

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
Name: tweet_id, dtype: int64

In [24]:
tweets_raw['in_reply_to_status_id'].value_counts().head()

6.671522e+17    2
8.562860e+17    1
8.131273e+17    1
6.754971e+17    1
6.827884e+17    1
Name: in_reply_to_status_id, dtype: int64

### Image predictions

In [25]:
image_predictions_raw

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [26]:
image_predictions_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [27]:
image_predictions_raw['tweet_id'].value_counts().head()

685532292383666176    1
826598365270007810    1
692158366030913536    1
714606013974974464    1
715696743237730304    1
Name: tweet_id, dtype: int64

### Extended tweets

In [28]:
extended_tweets_raw

,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,source,text,tweet_id,user_id
0,1563,NaN,NaN,614,"<a href=""http://twitter.com/download/iphone"" r...",Rare submerged pup here. Holds breath for a lo...,674632714662858753,4196983835
1,2965,NaN,NaN,1009,"<a href=""http://twitter.com/download/iphone"" r...",This is Tug. He's not required to wear the con...,678774928607469569,4196983835
2,24293,NaN,NaN,4901,"<a href=""http://twitter.com/download/iphone"" r...",This is Kevin. He's just so happy. 13/10 what ...,884562892145688576,4196983835
3,1264,NaN,NaN,523,"<a href=""http://twitter.com/download/iphone"" r...",*screams for a little bit and then crumples to...,670717338665226240,4196983835
4,31014,NaN,NaN,9504,"<a href=""http://twitter.com/download/iphone"" r...",We usually don't rate polar bears but this one...,845812042753855489,4196983835
5,903,NaN,NaN,381,"<a href=""http://twitter.com/download/iphone"" r...",This is Bubba. He's a Titted Peebles Aorta. Ev...,672160042234327040,4196983835
6,1962,NaN,NaN,1149,"<a href=""http://twitter.com/download/iphone"" r...",This is Erik. He's fucken massive. But also ki...,667886921285246976,4196983835
7,2404,NaN,NaN,964,"<a href=""http://twitter.com/download/iphone"" r...",This is Pepper. She's not fully comfortable ri...,675845657354215424,4196983835
8,240,NaN,NaN,59,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a rare Hungarian Pinot named Jessiga. ...,667773195014021121,4196983835
9,1692,NaN,NaN,386,"<a href=""http://twitter.com/download/iphone"" r...",This is Ralphson. He's very confused. Wonderin...,704347321748819968,4196983835


In [29]:
extended_tweets_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967 entries, 0 to 1966
Data columns (total 8 columns):
favorite_count           1967 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
retweet_count            1967 non-null int64
source                   1967 non-null object
text                     1967 non-null object
tweet_id                 1967 non-null int64
user_id                  1967 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 123.0+ KB


In [30]:
extended_tweets_raw['tweet_id'].value_counts().head()

685532292383666176    1
743510151680958465    1
805487436403003392    1
672466075045466113    1
685315239903100929    1
Name: tweet_id, dtype: int64

In [31]:
extended_tweets_raw['in_reply_to_status_id'].value_counts().head()

6.671522e+17    2
8.558181e+17    1
6.753494e+17    1
6.747934e+17    1
6.747522e+17    1
Name: in_reply_to_status_id, dtype: int64

In [32]:
extended_tweets_raw['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     1928
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       28
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [33]:
extended_tweets_raw['user_id'].value_counts()

4196983835    1967
Name: user_id, dtype: int64

### Users

In [34]:
users_raw.head()

,favourites_count,followers_count,friends_count,listed_count,statuses_count,user_id
0,134004,6889373,8,4389,7100,4196983835
1,134004,6889502,8,4475,7100,4196983835
2,134004,6889503,8,4477,7100,4196983835
3,134004,6889443,8,4414,7100,4196983835
4,134004,6889431,8,4413,7100,4196983835


In [35]:
users_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967 entries, 0 to 1966
Data columns (total 6 columns):
favourites_count    1967 non-null int64
followers_count     1967 non-null int64
friends_count       1967 non-null int64
listed_count        1967 non-null int64
statuses_count      1967 non-null int64
user_id             1967 non-null int64
dtypes: int64(6)
memory usage: 92.3 KB


### Problems

- Tidiness:
    - In tweets archive dog "stages" are datapoints, but should be observations (and categorical);
    - tweets archive, extended tweets and image pradictions should be single dataset;
    - (alternatively twitter related information in tweets archive and extended tweets could go into one dataset and dog related information in tweets archive and image predictions could go to another as this could be seen as two separate concenrs. In our case I think all this is part of same observation, so I decided against splitting data into two datasets);

- Quality:
    - tweets archive:
        - `retweet_status_id` and `retweeted_status_user_id` are in scientific notation (also float64);
        - False dog names (captured in `not_names`);
        - Columns `name`, `doggo`, `floofer`, `pupper`, `puppo` poluted with 'None' instead of np.NaN;
        - `source` would be fine as just inner text of anchor tag;
        - `source` could be categorical;
        - `timestamp` and `retweeted_status_timestamp` are objects, but represent datetimes;
    - extended_tweets:
        - `in_reply_to_status_id` and `in_reply_to_user_id` should be integers (or at least be represented in non-scientific notation);
        - `source` has same issues as in tweets archive;
        - `user_id` has single unique value;
    - users:
        - Turns out all tweets came from one user. This dataset is useless;

## Clean

### Define

- In columns `name`, `doggo`, `floofer`, `pupper`, `puppo` replace 'None' with np.NaN;
- Melt `doggo`, `floofer`, `pupper`, `puppo` column into `stage` column;
- Merge tweet archive, extended tweets and image predictions into single dataset;
- Replace false dog names (captured in `not_names`) with np.NaN;
- Extract inner text of anchor tag in `source` and replace the value with it;
- Convert `source` to category;
- Convert `timestamp` and `retweeted_status_timestamp` to datetime;
- Convert `in_reply_to_status_id` and `in_reply_to_user_id` to strings (I'd rather have them as integers, but we [can't have NAs in integer series](http://pandas.pydata.org/pandas-docs/stable/gotchas.html#support-for-integer-na), and these are identifier values, so representation is important);
- Drop `user_id`;

### Act

In [36]:
tweets = tweets_raw.copy()
extended_tweets = extended_tweets_raw.copy()
image_predictions = image_predictions_raw.copy()

#### In columns name, doggo, floofer, pupper, puppo replace 'None' with np.NaN

In [37]:
tweets[['name', 'doggo', 'floofer', 'pupper', 'puppo']] = (
    tweets[['name', 'doggo', 'floofer', 'pupper', 'puppo']]
    .replace('None', np.NaN))

In [38]:
tweets

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN,NaN,NaN,NaN
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,NaN,NaN,NaN,NaN,NaN
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,NaN,NaN,NaN,NaN
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,NaN,NaN,NaN,NaN,NaN
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,NaN,NaN,NaN,NaN
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,NaN,NaN,NaN


#### Melt doggo, floofer, pupper, puppo column into stage column

In [39]:
dog_stages = tweets[['doggo', 'floofer', 'pupper', 'puppo']]
tweets['stage'] = (
    dog_stages
    .notnull()
    .idxmax(1)
    .pipe(lambda row: dog_stages.lookup(row.index, row.values)))

In [40]:
tweets[tweets['stage'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,stage
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,NaN,NaN,NaN,doggo
12,889665388333682689,NaN,NaN,2017-07-25 01:55:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo that seems to be on the fence a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889665388...,13,10,NaN,NaN,NaN,NaN,puppo,puppo
14,889531135344209921,NaN,NaN,2017-07-24 17:02:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Stuart. He's sporting his favorite fan...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889531135...,13,10,Stuart,NaN,NaN,NaN,puppo,puppo
29,886366144734445568,NaN,NaN,2017-07-15 23:25:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Roscoe. Another pupper fallen victim t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/886366144...,12,10,Roscoe,NaN,NaN,pupper,NaN,pupper
43,884162670584377345,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,NaN,NaN,NaN,doggo
46,883360690899218434,NaN,NaN,2017-07-07 16:22:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Grizzwald. He may be the floofiest floofe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/883360690...,13,10,Grizzwald,NaN,floofer,NaN,NaN,floofer
49,882762694511734784,NaN,NaN,2017-07-06 00:46:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gus. He's quite the cheeky pupper. Alr...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/882762694...,12,10,Gus,NaN,NaN,pupper,NaN,pupper
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,NaN,NaN,pupper,NaN,pupper
71,878776093423087618,NaN,NaN,2017-06-25 00:45:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Snoopy. He's a proud #PrideMonthPuppo....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/878776093...,13,10,Snoopy,NaN,NaN,NaN,puppo,puppo
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,NaN,NaN,NaN,pupper,NaN,pupper


In [41]:
tweets = tweets.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

#### Merge tweet archive, extended tweets and image predictions into single dataset

In [42]:
set(tweets.columns) & set(extended_tweets.columns)

{'in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'text', 'tweet_id'}

In [43]:
# I trust 'text', in_reply_to_status_id', 'in_reply_to_user_id', 'source' values more in dataset I pulled myself
# so I'll keep the ones from extended tweets
tweets = tweets.drop(['text', 'in_reply_to_status_id', 'in_reply_to_user_id', 'source'], axis = 1)

In [44]:
set(tweets.columns) & set(image_predictions.columns)

{'tweet_id'}

In [45]:
set(extended_tweets.columns) & set(image_predictions.columns)

{'tweet_id'}

In [46]:
master = (
    extended_tweets
    .merge(tweets, how='left', on='tweet_id')
    .merge(image_predictions, how='left', on='tweet_id'))

In [47]:
master

,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,source,text,tweet_id,user_id,timestamp,retweeted_status_id,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,1563,NaN,NaN,614,"<a href=""http://twitter.com/download/iphone"" r...",Rare submerged pup here. Holds breath for a lo...,674632714662858753,4196983835,2015-12-09 16:52:27 +0000,NaN,...,1.0,jellyfish,0.432748,False,goldfish,0.113111,False,coral_reef,0.087047,False
1,2965,NaN,NaN,1009,"<a href=""http://twitter.com/download/iphone"" r...",This is Tug. He's not required to wear the con...,678774928607469569,4196983835,2015-12-21 03:12:08 +0000,NaN,...,1.0,Pembroke,0.194681,True,toy_poodle,0.121821,True,Pomeranian,0.096843,True
2,24293,NaN,NaN,4901,"<a href=""http://twitter.com/download/iphone"" r...",This is Kevin. He's just so happy. 13/10 what ...,884562892145688576,4196983835,2017-07-11 00:00:02 +0000,NaN,...,1.0,pug,0.546406,True,French_bulldog,0.404291,True,Brabancon_griffon,0.044002,True
3,1264,NaN,NaN,523,"<a href=""http://twitter.com/download/iphone"" r...",*screams for a little bit and then crumples to...,670717338665226240,4196983835,2015-11-28 21:34:09 +0000,NaN,...,1.0,Pomeranian,0.368161,True,Pekinese,0.350973,True,golden_retriever,0.114902,True
4,31014,NaN,NaN,9504,"<a href=""http://twitter.com/download/iphone"" r...",We usually don't rate polar bears but this one...,845812042753855489,4196983835,2017-03-26 01:38:00 +0000,NaN,...,1.0,Samoyed,0.979803,True,chow,0.015923,True,white_wolf,0.001303,False
5,903,NaN,NaN,381,"<a href=""http://twitter.com/download/iphone"" r...",This is Bubba. He's a Titted Peebles Aorta. Ev...,672160042234327040,4196983835,2015-12-02 21:06:56 +0000,NaN,...,1.0,pug,0.561027,True,French_bulldog,0.222114,True,Labrador_retriever,0.065456,True
6,1962,NaN,NaN,1149,"<a href=""http://twitter.com/download/iphone"" r...",This is Erik. He's fucken massive. But also ki...,667886921285246976,4196983835,2015-11-21 02:07:05 +0000,NaN,...,1.0,Pomeranian,0.800432,True,Pekinese,0.168445,True,Chihuahua,0.008950,True
7,2404,NaN,NaN,964,"<a href=""http://twitter.com/download/iphone"" r...",This is Pepper. She's not fully comfortable ri...,675845657354215424,4196983835,2015-12-13 01:12:15 +0000,NaN,...,1.0,pug,0.883952,True,Boston_bull,0.011057,True,French_bulldog,0.009840,True
8,240,NaN,NaN,59,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a rare Hungarian Pinot named Jessiga. ...,667773195014021121,4196983835,2015-11-20 18:35:10 +0000,NaN,...,1.0,West_Highland_white_terrier,0.360465,True,pug,0.093494,True,ice_bear,0.069038,False
9,1692,NaN,NaN,386,"<a href=""http://twitter.com/download/iphone"" r...",This is Ralphson. He's very confused. Wonderin...,704347321748819968,4196983835,2016-02-29 16:47:42 +0000,NaN,...,1.0,teddy,0.233378,False,feather_boa,0.088474,False,Brittany_spaniel,0.082917,True


#### Replace false dog names (captured in not_names) with np.NaN

In [48]:
master['name'] = np.where(np.isin(master['name'], not_names), np.NaN, master['name'])

In [49]:
master[np.isin(master['name'], not_names)]

,favorite_count,in_reply_to_status_id,in_reply_to_user_id,retweet_count,source,text,tweet_id,user_id,timestamp,retweeted_status_id,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


#### Extract inner text of anchor tag in source and replace the value with it

In [50]:
master['source'] = master['source'].str.extract(r'<a .+?>(.+)</a>', expand=False)

In [51]:
master['source'].value_counts()

Twitter for iPhone    1928
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

#### Convert source to category

In [52]:
master['source'] = master['source'].astype('category')

#### Convert timestamp and retweeted_status_timestamp to datetime

In [53]:
master['timestamp'] = pd.to_datetime(master['timestamp'])
master['retweeted_status_timestamp'] = pd.to_datetime(master['retweeted_status_timestamp'])

#### Convert in_reply_to_status_id and in_reply_to_user_id to strings

In [54]:
in_reply_to_status_ids_tmp = master.apply(
    lambda row: '%.0f' % row['in_reply_to_status_id'],
    axis=1)
master['in_reply_to_status_id'] = np.where(
    in_reply_to_status_ids_tmp == 'nan',
    np.NaN,
    in_reply_to_status_ids_tmp)

In [55]:
in_reply_to_user_ids_tmp = master.apply(
    lambda row: '%.0f' % row['in_reply_to_user_id'],
    axis=1)
master['in_reply_to_user_id'] = np.where(
    in_reply_to_user_ids_tmp == 'nan',
    np.NaN,
    in_reply_to_user_ids_tmp)

#### Drop user_id

In [56]:
master = master.drop(['user_id'], axis=1)

In [57]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967 entries, 0 to 1966
Data columns (total 27 columns):
favorite_count                1967 non-null int64
in_reply_to_status_id         23 non-null object
in_reply_to_user_id           23 non-null object
retweet_count                 1967 non-null int64
source                        1967 non-null category
text                          1967 non-null object
tweet_id                      1967 non-null int64
timestamp                     1966 non-null datetime64[ns]
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 1966 non-null object
rating_numerator              1966 non-null float64
rating_denominator            1966 non-null float64
name                          1328 non-null object
stage                         300 non-null object
jpg_url                       1966 non-null object
img_num              